#Basic#


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

#Image classification#

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image

###Define Model###

In [ ]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer = tf.keras.optimizers.RMSprop(0.001), metrics=['accuracy'])

### Data Augmentation ###
https://docs.google.com/document/d/1RFU4Z2QCy2HNVZvRyJQsv6UAKY9dzJ8lYYlebNYKl2c/edit?usp=sharing

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip= True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(300,300), batch_size=128, class_mode='binary')

### Transfer Learning ###
https://docs.google.com/document/d/1jkLyg-5JVlrehJ_lnHd2QJQHjhGjlS2gp8p6aWUXZXs/edit?usp=sharing

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model

pre_trained_model = VGG16(input_shape=(600,600,3), include_top = False)
for layer in pre_trained_model.layers: 
  layer.trainable = False

pre_output = pre_trained_model.output   

x = Flatten()(pre_output)
x = Dense(512, activation='relu')(x)
x = Dense(5, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

### Convert Image to Array ###
https://docs.google.com/document/d/1HlYpvJ6rNuiU7yFHaMemU29s9c79CO_LVlXaGoLtHRw/edit?usp=sharing



In [ ]:
img = Image.open('this.jpg')
#Resize the image
img = img.resize((300,300))
#If you want it to be grayscale, add a .convert('L') behind
# img = img.resize((300,300)).convert('L')
data = np.array(img)
#Since PIL returns a color channel of 4 (RGBA where A is the transparency) but you want 3 color channels (RBG), you can just slice the A channel off
#Change from (300,300,4) to (300,300,3)
data = data[:,:,:3]


#NLP#
https://docs.google.com/document/d/1iSlxYepQsTZXbSkYPT_1SNnDc_IJc9WRY5ynIGnn7ag/edit?usp=sharing

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

###Define Model###

In [ ]:
model = Sequential([
                    Embedding(input_dim=num_vocab+1, output_dim = 100, input_length=mlen-1),
                    Bidirectional(LSTM(150)),
                    Dense(num_vocab+1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])

###Get text corpus from .txt file###

In [ ]:
wFile = open('irish-lyrics.txt', 'r').read()
corpus = wFile.lower().split('\n')

###Fit Tokenizer###

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_idx = tokenizer.word_index
rev_word_idx = {word_idx[i]:i for i in list(word_idx)}

###Create nGrams###

In [ ]:
ngrams = []

for line in corpus:
  enc = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(enc)):
    ngram = enc[:i+1]
    ngrams.append(ngram)

###Pad  nGrams###

In [ ]:
mlen = max([len(x) for x in ngrams])
padded = pad_sequences(ngrams, maxlen = mlen, padding = 'pre', truncating='pre')
padded = np.array(padded)

###Get X and Y data from nGrams###


In [ ]:
x = padded[:,:-1]
y = padded[:,-1]

###OHE y data###

In [ ]:
y = tf.keras.utils.to_categorical(y,num_classes=num_vocab + 1)

#Time series#

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.data import Dataset

###Learning Rate Scheduler###
https://docs.google.com/document/d/1akY5R0-BEr4BvJPkfoz_P3_XJ6nrCoxEhHA0LlO6mJY/edit?usp=sharing

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch/20))
history = model.fit(dataset, epochs=100, callbacks=[lr_schedule], verbose=0)

In [ ]:
plt.semilogx(history.history['lr'],history.history['loss'])
plt.axis([1e-8, 1e-3, 0, 300])

###Creating Data Set###
https://colab.research.google.com/drive/14e2-UmfV2FGMmWCsp405NKXqDbzunBKZ?usp=sharing


In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer_size):
  ds = Dataset.from_tensor_slices(series)
  ds = ds.window(window_size+1, step=1,drop_remainder=True)
  ds = ds.flat_map(lambda window: window.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer_size).map(lambda data: (data[:-1], data[-1:]))
  ds = ds.batch(batch_size).prefetch(1)

  return ds

###Predicting values###

In [ ]:
forecast = []

for time in range(len(series)- window_size):
  tmp = series[time:time+window_size]
  tmp = np.expand_dims(tmp, axis=0)
  forecast.append(model.predict(tmp))

val_forecast = np.array(forecast[split_time-window_size:])

val_pred = val_forecast[:,0,0]

###Metrics###

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, val_pred).numpy()

###Conv1D Architecture###

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

#Download Files#

###Normal wget###

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /content/horse-or-human.zip

###Download from Dropbox###

In [ ]:
def dropbox_dl(link):
  new = link.replace('www.dropbox.com', 'dl.dropboxusercontent.com')
  return new

new = dropbox_dl('https://www.dropbox.com/s/dzie78zcgy1r0ah/train-hands.zip?dl=0')

print(new)

In [ ]:
!wget --no-check-certificate \
    #Replace the link below with the printed string from the previous cell (Value of new)
    https://dl.dropboxusercontent.com/s/dzie78zcgy1r0ah/train-hands.zip?dl=0 \
    -O /content/here.zip

#Callbacks#

In [ ]:
class mycallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.95):
      self.model.stop_training = True
      print('95% accuracy reached')

callback = mycallback()

#Preprocessing#


###Zip Files###

In [ ]:
#Unzip files

train_zip = '/content/horse-or-human.zip'
train_dir = '/content/horse-or-human'


with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(train_dir)

###.Txt Files###

In [ ]:
#Read .txt files

wFile = open('irish-lyrics.txt', 'r').read()

#Plot acc graph#

In [ ]:
def plot(model, metric):
  met = model.history[metric]
  plt.plot(met)
  plt.xlabel('epochs')
  plt.ylabel(metric)
  plt.show()

plot(history, 'accuracy')

#Uploading files for prediction Colab#

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  model.predict(fn)

In [ ]:
from google.colab import files
x = files.upload()

#Save and Load Model#

In [ ]:
#Save model
model.save('model.h5')

#Load model
model = tf.keras.models.load_model('model.h5')

#Reset all States in Keras#

In [ ]:
tf.keras.backend.clear_session()

#Pandas Dataframe#


###Missing Values###

In [ ]:
#See number of missing values per column in the df
df.isnull().sum()

#Show the rows where the value of 'Age' column is 0.0
df.loc[df['Age'] == 0.0]

#Show the rows where the value of 'Embarked' column is null
df.loc[df['Embarked'].isnull()]

#Drop rows with missing values. inplace means whether to modify the df itself, or to just return the modified df without actually applying the modifications to the df
df.dropna(inplace=True)

#Fill rows with missing values, with the mean of that row
df.fillna(df.mean(), inplace=True)

###One Hot Encode###

In [ ]:
#One Hot Encode the Gender column
df['Gender'] = pd.get_dummies(df['Gender'])

###Convert predictions to csv file###

In [ ]:
preds = rclf.predict(x_test)
pred_csv = pd.DataFrame(preds, columns=['Survived']).to_csv('preds.csv')